In [1]:
# Import Libraries
import os
import glob
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.dates as mdates

## Load Data

In [2]:
# Read processed file and print top rows
df = pd.read_csv('..\\..\\data\\processed\\weather\\noaa\\nycHistorical.csv', \
                 index_col='DATE', parse_dates=True)
df.head()

C:\Users\jain432\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,TempVals,TempCodes,DewVals,DewCodes,SkyCoverVals,SkyCoverCodes,WindSpeedVals,WindSpeedCodes,StationID,StationLocation
DATE,,,,,,,,,,
2005-01-01 00:51:00,120,1,40,1,NaN,99,82.0,1,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 01:51:00,120,1,50,1,NaN,99,NaN,99,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 02:51:00,120,1,50,1,NaN,99,108.0,1,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 03:51:00,120,1,60,1,NaN,99,93.0,1,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 04:51:00,110,1,60,1,NaN,99,NaN,99,72505394728,"NY CITY CENTRAL PARK, NY US"


In [3]:
# Scaling of temperature values is requried as per the documentation
df['TempVals'] = df['TempVals'] / 10.0 
df['DewVals'] = df['DewVals'] / 10.0 

# Remove noisy values and print top rows of processed data frame
df = df[(df['TempVals'] < 999) & (df['DewVals'] < 999)].copy()
df.head()

,TempVals,TempCodes,DewVals,DewCodes,SkyCoverVals,SkyCoverCodes,WindSpeedVals,WindSpeedCodes,StationID,StationLocation
DATE,,,,,,,,,,
2005-01-01 00:51:00,12.0,1,4.0,1,NaN,99,82.0,1,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 01:51:00,12.0,1,5.0,1,NaN,99,NaN,99,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 02:51:00,12.0,1,5.0,1,NaN,99,108.0,1,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 03:51:00,12.0,1,6.0,1,NaN,99,93.0,1,72505394728,"NY CITY CENTRAL PARK, NY US"
2005-01-01 04:51:00,11.0,1,6.0,1,NaN,99,NaN,99,72505394728,"NY CITY CENTRAL PARK, NY US"


# Data Analytics

In [4]:
# Task: for each month, compute the median temperature (indegC) and dew point values (in degC).
# Why?: 
# As expected, the median temperature follows a bell curve over a calendar year.
# The information is helpful in putting each month in one of the following categories:
# Cold: <5degC; Cool: 5degC-10degC; Warm: 10degC-20degC; Hot: >20degC.
# Accordingly, 
# Jan, Feb, and Dec are Cold months,
# Mar and Nov are Cool months,
# Apr, May, and Oct are Warm months, and
# Jun, Jul, Aug, Sep are the Hot months
dfValidVals = df[['TempVals', 'DewVals']]
dfGrouped = dfValidVals.groupby([dfValidVals.index.year, \
                                 dfValidVals.index.month]).median().rename_axis(index=["Year", \
                                                                                       "Month"]).reset_index()

dfGrouped[["Month", "TempVals", "DewVals"]].groupby("Month").median()

,TempVals,DewVals
Month,,
1,1.35,-5.80
2,1.85,-4.20
3,5.00,-3.05
4,11.05,3.00
5,16.00,11.10
6,21.10,16.00
7,24.40,19.00
8,23.60,18.35
9,21.05,14.20


### Deeper Analysis
1. Time of Day pricing from ConEdison further validates our above-mentioned categorisation. In the hot months (June-September), the time of day price during the day (8AM till midnight) is highest (23.07 cents/kWh) [ref - https://www.coned.com/en/save-money/energy-saving-programs/time-of-use].

2. Why we need this categorisation? - To recommend set point temperature for the thermostat, it is important that we are aware of internal load. One important parameter for internal load is the clothing level which depends on the season. As of now, we are using CBE Thermal Comfort Tool to compute the clothing level [ref-https://comfort.cbe.berkeley.edu/]. For any particular day, they use temperature at 6 AM to predict dynamic clothing level. Based on the median temperature at 6 AM for each month, we used the tool to compute clothing level for each season, which is as follows: 

        Cold (0.78 clo) > Cool (0.63 clo) > Warm (0.56 clo) > Hot (0.5 clo).


3. Another important parameter denoting the internal load is the metabolic activity, basically the heat generated by the occupants due to their activities within the home. We don't want to deploy sensors on the user, or heavily instrument the home. We are okay with an approximation. Our hypothesis is that data from occupancy sensor when combined with meta data (such as type of occupants), can provide us a good estimate of human activity over the day. Once we know the intensity of human activity, we can get an estimate of internal load due to human activities. 
    
4. Since we don't have occupancy sensor installed in the homes, we are using ASHRAE table to compute metabolic activity depending on the type of occupants living in that apartment.

        Married Couple w kids (2.0 met) > Married Couple wo kids (1.7 met) > Single Bachelor (1.3 met) > Old Couple wo kids (1.0 met)
        

5. I do see a challenge even in estimating occupants' activity based on single motion sensor. The questions are: how many occupancy sensors we might need? where shall we install those sensors? can we use additional information from other sensors (such as smart meter) to minimise the number of occupancy sensors and accurately estimate occupants' activity level? We must do a literature survey to do this estimation.  

In [5]:
# Task: 
# For each month, compute median temperature and dew point temperature for every hour of the day
# Why?:  
# Our goal is to understand typical temperature variation across a day for different settings and 
# then recommend set-point, lighting, and appliance settings to the user for every hour of the day. 
# Therefore, here we are doing this computation. This data will be used for the web-tool
dfHourlyVariations = dfValidVals.groupby([dfValidVals.index.month, \
                                          dfValidVals.index.hour]).agg(['mean', \
                                                                        'median', \
                                                                        'std']).rename_axis(index=["Month", \
                                                                                                   "Hour"])
dfHourlyVariations.columns = ['_'.join(col) for col in dfHourlyVariations.columns]
dfHV = dfHourlyVariations.reset_index()

In [6]:
# Task: 
# Compute confidence interval for air temperature and dew point temperature
# Why?:  
# We need this information for data plotting purposes on the web tool
dfHV['TempVals_cileft'] = dfHV['TempVals_mean'] - 1.96 * dfHV['TempVals_std']
dfHV['TempVals_ciright'] = dfHV['TempVals_mean'] + 1.96 * dfHV['TempVals_std']

dfHV['DewVals_cileft'] = dfHV['DewVals_mean'] - 1.96 * dfHV['DewVals_std']
dfHV['DewVals_ciright'] = dfHV['DewVals_mean'] + 1.96 * dfHV['DewVals_std']

dfHV.head()

,Month,Hour,TempVals_mean,TempVals_median,TempVals_std,DewVals_mean,DewVals_median,DewVals_std,TempVals_cileft,TempVals_ciright,DewVals_cileft,DewVals_ciright
0,1,0,1.577132,2.0,5.575997,-5.388760,-5.6,8.032642,-9.351823,12.506087,-21.132737,10.355218
1,1,1,1.297070,1.7,5.676198,-5.593555,-6.0,8.004704,-9.828279,12.422419,-21.282774,10.095665
2,1,2,1.209438,1.7,5.663687,-5.515663,-6.0,8.041807,-9.891390,12.310265,-21.277604,10.246279
3,1,3,1.056514,1.4,5.678813,-5.237852,-5.0,8.050142,-10.073959,12.186988,-21.016131,10.540427
4,1,4,1.088281,1.7,5.943634,-5.217188,-5.3,8.284663,-10.561242,12.737805,-21.455126,11.020751


In [7]:
# Save hourly temperature, dew-point, and the confidence interval data to a CSV
dfHV.to_csv("..\\..\\data\\processed\\weather\\noaa\\nycMonthlyHoD.csv", index=False)

In [8]:
# Task: 
# For every month, compute median air temperature for each hour
# Why?:  
# Generic data exploration
dfValidTemp = df['TempVals']
dfHourlyTemp = dfValidTemp.groupby([dfValidTemp.index.hour, \
                                    dfValidTemp.index.month]).median().rename_axis(index=["Hour", \
                                                                                          "Month"]).unstack()
dfHourlyTemp.head()

Month,1,2,3,4,5,6,7,8,9,10,11,12
Hour,,,,,,,,,,,,
0,2.0,2.8,6.0,11.0,16.0,21.1,24.0,23.9,21.1,15.6,9.4,5.0
1,1.7,1.4,5.0,11.0,15.6,21.1,24.0,23.9,21.0,15.0,9.0,4.4
2,1.7,1.1,5.3,11.0,15.0,21.0,23.9,23.0,21.0,15.0,9.0,4.4
3,1.4,2.0,4.4,9.4,15.0,20.3,23.3,22.8,20.6,14.4,9.0,4.4
4,1.7,1.1,4.0,10.0,14.4,20.6,23.0,22.8,20.0,14.0,8.3,3.9


In [9]:
# Task: 
# For each season, compute median air temperature for each hour
# Why?:  
# Generic data exploration 
dfHourlyTempVal = pd.concat([dfHourlyTemp[[1, 2, 12]].median(axis=1), \
                      dfHourlyTemp[[3, 11]].median(axis=1), \
                      dfHourlyTemp[[4, 5, 10]].median(axis=1), \
                      dfHourlyTemp[[6, 7, 8, 9]].median(axis=1)], axis=1)
dfHourlyTempVal.columns = ['Cold', 'Cool', 'Warm', 'Hot']
dfHourlyTempVal.head()

,Cold,Cool,Warm,Hot
Hour,,,,
0,2.8,7.70,15.6,22.5
1,1.7,7.00,15.0,22.5
2,1.7,7.15,15.0,22.0
3,2.0,6.70,14.4,21.7
4,1.7,6.15,14.0,21.7


In [10]:
# Saving data for hourly median temperature for each season
dfHourlyTempVal.to_csv("..\\..\\data\\processed\\weather\\noaa\\nycHourlyTemp.csv")

In [11]:
# Task: 
# For every month, compute median dew-point temperature for each hour
# Why?:  
# Generic data exploration
dfValidDew = df['DewVals']
dfHourlyDew = dfValidDew.groupby([dfValidDew.index.hour, \
                                  dfValidDew.index.month]).median().rename_axis(index=["Hour", \
                                                                                       "Month"]).unstack()
dfHourlyDew.head()

Month,1,2,3,4,5,6,7,8,9,10,11,12
Hour,,,,,,,,,,,,
0,-5.6,-4.4,-3.0,3.0,9.40,16.0,19.0,18.30,15.6,10.0,1.4,-2.00
1,-6.0,-4.7,-2.2,3.0,10.00,16.0,19.0,18.15,15.0,10.0,2.2,-1.85
2,-6.0,-4.0,-2.0,3.0,10.60,16.0,19.0,18.30,15.0,10.0,2.8,-1.70
3,-5.0,-4.0,-2.2,3.3,10.05,15.6,19.0,18.30,15.0,10.0,2.2,-1.70
4,-5.3,-5.0,-2.8,3.3,10.60,16.0,19.0,18.30,15.0,9.4,2.0,-2.00


In [12]:
# Task: 
# For each season, compute median dew-point temperature for each hour
# Why?:  
# Generic data exploration
dfHourlyDewVal = pd.concat([dfHourlyDew[[1, 2, 12]].median(axis=1), \
                            dfHourlyDew[[3, 11]].median(axis=1), \
                            dfHourlyDew[[4, 5, 10]].median(axis=1), \
                            dfHourlyDew[[6, 7, 8, 9]].median(axis=1)], axis=1)
dfHourlyDewVal.columns = ['Cold', 'Cool', 'Warm', 'Hot']
dfHourlyDewVal.head()

,Cold,Cool,Warm,Hot
Hour,,,,
0,-4.4,-0.8,9.4,17.150
1,-4.7,0.0,10.0,17.075
2,-4.0,0.4,10.0,17.150
3,-4.0,0.0,10.0,16.950
4,-5.0,-0.4,9.4,17.150


In [13]:
# Saving data for hourly median dew-point temperature for each season
dfHourlyDewVal.to_csv("..\\..\\data\\processed\\weather\\noaa\\nycHourlyDew.csv")